In [1]:
import pandas as pd
import numpy as np

# Load the CSV
df = pd.read_csv("sample_data_year.csv", low_memory=False)

# Convert Time column
df['Time'] = pd.to_datetime(df['Time'], format='%d-%m-%Y %H:%M', errors='coerce')
df = df.dropna(subset=['Time']).sort_values('Time').reset_index(drop=True)

# Convert numeric columns
cols_to_clean = ['FO_ME_Cons', 'Ship_Speed', 'CppPitch', 'Wind_Speed', 'HEEL', 'Fore_Draft', 'Aft_Draft']
for col in cols_to_clean:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Compute ME fuel consumption per minute (GE is ignored)
df['Fuel_Liters'] = df['FO_ME_Cons'].diff().clip(lower=0)

# Clean missing values
df.fillna(method='ffill', inplace=True)
df.dropna(inplace=True)

# Trim & Average Draft
df['Trim'] = df['Aft_Draft'] - df['Fore_Draft']
df['Avg_Draft'] = (df['Fore_Draft'] + df['Aft_Draft']) / 2

# Distance per minute
df['Distance_NM'] = df['Ship_Speed'] / 60

# CO2 emissions (only from FO_ME_Cons)
FUEL_DENSITY = 0.991  # kg/L
CO2_FACTOR = 3.114    # g CO₂/g fuel
fuel_kg = df['Fuel_Liters'] * FUEL_DENSITY
df['CO2_Emission_g'] = fuel_kg * 1000 * CO2_FACTOR

# Remove outliers using IQR
def remove_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    return data[(data[column] >= Q1 - 1.5 * IQR) & (data[column] <= Q3 + 1.5 * IQR)]

for col in ['Fuel_Liters', 'Distance_NM', 'Ship_Speed']:
    df = remove_outliers_iqr(df, col)

# Remove ship at rest (speed < 1 knot)
df = df[df['Ship_Speed'] > 1]

# ✅ REMOVE High Wind Speed Data (Wind_Speed > 30 m/s)
df = df[df['Wind_Speed'] <= 30]

# -------------------------
# 📌 Final CII Calculation
# -------------------------
GT = 14052  # Ship's gross tonnage
total_fuel = df['Fuel_Liters'].sum()
total_co2 = df['CO2_Emission_g'].sum()
total_distance = df['Distance_NM'].sum()
cii_full_year = total_co2 / (GT * total_distance)

# -------------------------------
# 📌 Weekly Estimate CII Method
# -------------------------------
df['Month'] = df['Time'].dt.month
df['Week'] = df['Time'].dt.isocalendar().week

# Filter to April and May (month 4 and 5 only) for stable conditions
filtered_df = df[df['Month'].isin([4, 5])]

# Choose one week with at least 5 days of data
weekly_df = filtered_df.groupby('Week').filter(lambda x: len(x) >= 60 * 24 * 5)

if not weekly_df.empty:
    sample_week = weekly_df['Week'].unique()[0]
    one_week_data = weekly_df[weekly_df['Week'] == sample_week]

    fuel_week = one_week_data['Fuel_Liters'].sum()
    co2_week = one_week_data['CO2_Emission_g'].sum()
    distance_week = one_week_data['Distance_NM'].sum()

    # Scale to annual
    cii_estimated_from_week = (co2_week * 52) / (GT * distance_week * 52)
else:
    cii_estimated_from_week = np.nan

# -------------------------
# ✅ Print Results
# -------------------------
print("=" * 40)
print("✅ Final Cleaned CII Calculation Summary (Excluding High Wind)")
print("=" * 40)
print(f"Duration: {df['Time'].iloc[0]} to {df['Time'].iloc[-1]}")
print(f"Data points: {len(df)}")
print(f"Total Fuel (L): {total_fuel:,.2f}")
print(f"Total CO₂ (g): {total_co2:,.2f}")
print(f"Total Distance (NM): {total_distance:,.2f}")
print(f"Final Annual CII (Full Year): {cii_full_year:.4f}")
print(f"Estimated CII (1 Week × 52): {cii_estimated_from_week:.4f}")
print("=" * 40)


C:\Users\siddh\AppData\Local\Temp\ipykernel_3000\4201764910.py:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


✅ Final Cleaned CII Calculation Summary (Excluding High Wind)
Duration: 2024-03-04 00:01:00 to 2025-03-29 11:49:00
Data points: 338358
Total Fuel (L): 8,407,883.00
Total CO₂ (g): 25,946,508,333.04
Total Distance (NM): 97,001.40
Final Annual CII (Full Year): 19.0354
Estimated CII (1 Week × 52): 20.2083
